In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import datetime
from datetime import timedelta
import sys
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from IPython.display import clear_output
from scipy.special import rel_entr
import matplotlib.cm as cm

In [ ]:
def code_to_date(datecode,timecode): #converts datetime code into datetime stamps 
  return pd.Timestamp( datetime(2009, 1, 1) + timedelta(int(datecode)) + timedelta(minutes=30*int(timecode)) )

def split(code): #splits datetime code into date,seconds
  s = [str(i) for i in str(code)]
  res1 = int("".join(s[:3]))
  res2 = int("".join(s[3:]))
  return res1,res2

def split_run(datetimecolumn): # returns a column of (date,time) 
  datecode = datetimecolumn
  timecode = []
  for i,v in enumerate(datecode):
    temp = split(v)
    datecode[i] = temp[0]
    timecode.append(temp[1])
  date_timecode = np.column_stack((datecode, timecode))
  return date_timecode

def code_to_date_run(datetimecode): # runs code_to_date function on dataset
  templist = []
  for i in datetimecode:
    templist.append(code_to_date(i[0],i[1]))
  return templist

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hlist = ['ID','Datetime','Energy']
df = pd.read_csv(r"/content/drive/My Drive/Summer Work and SPROJ/Unzipped/File1.txt",sep=" ",names=hlist)
# df = pd.read_csv("/content/drive/My Drive/File1.txt",sep=" ",names=hlist)

df = df[['Datetime','ID','Energy']]
df['Datetime'] = code_to_date_run(split_run(np.array(df['Datetime'])))
df.head()

,Datetime,ID,Energy
0,2009-07-15 01:30:00,1392,0.140
1,2009-07-15 02:00:00,1392,0.138
2,2009-07-15 02:30:00,1392,0.140
3,2009-07-15 03:00:00,1392,0.145
4,2009-07-15 03:30:00,1392,0.145


## Data Cleaning

In [ ]:
df['ID'] = df['ID'].astype('category')     # Converting ID column's type to categorical (Modelling Factor Datatype)
# Creating a new column in the dataframe which contains value counts
ID_counts = df['ID'].value_counts().rename('ID_counts')            

df = df.merge(ID_counts.to_frame(),left_on='ID',right_index=True)      # Merging the new column wrt the ID column

df = df[df['ID_counts'] >= 1440] # Filtering the data s.t the ID's of the retained data occurs 1440 or more times

df.drop({'ID_counts'},axis=1,inplace=True)    # Dropping the column added

null_columns = df.columns[df.isnull().any()]    # Finding columns which contain null values
df[null_columns].isnull().sum()                 # Finding the sum of all the null values

duplicates = df[df.duplicated(keep=False)]      #Finding entries which have exactly the same value for all 3 columns
df.drop_duplicates(keep=False,inplace=True)

In [ ]:
#@title
# list_of_IDs = pd.DataFrame(df['ID'].unique())     #Getting all Unique IDs
# list_of_IDs = np.array(list_of_IDs.sample(20))     #Randomly selecting N of them

In [ ]:
#@title
# list_of_IDs

In [ ]:
#@title
# df_small = pd.DataFrame(columns=['Datetime','ID','Energy'])                 # Making a new Dataframe with smaller size using list_of_IDs
# for id in list_of_IDs:
#   df_small = df_small.append(df[ df['ID'] == id[0] ], ignore_index=True)     

In [ ]:
#@title
# print(df.shape)         # Size Differences
# print(df_small.shape)

In [ ]:
#@title
# df2 = df.copy(deep=True)          #Making copy of DF
# df = df_small                     #Using the Small dataframe from here on

In [ ]:
#@title
# df = df2

In [ ]:
#@title
# df = df_small                     #Using the Small dataframe from here on

In [ ]:
#@title
# print(df.shape) # Shape of DataFrame
# print(df.ID.value_counts()) # ID count in DataFrame

In [ ]:
def fix_start_end(dataframe):
  new_df = pd.DataFrame(columns=['Datetime','ID','Energy'])
  ID_list = dataframe["ID"].unique()
  

  for index, ID in enumerate(ID_list):
    clear_output(wait=True)
    print(index,'/',len(ID_list))
    temp = dataframe[dataframe["ID"] == ID]
    temp.reset_index()
    for index,value in enumerate(temp['Datetime']):
      if value.hour == 0 and value.minute == 0:
        end_len = len(temp.loc[temp.Datetime > value])
        round_off = int(end_len/48)
        new_df = pd.concat([new_df,temp.loc[temp.Datetime > value][:round_off*48]])
        break

  return new_df

In [ ]:
df = fix_start_end(df)

987 / 988


In [ ]:
#@title
# test = fix_start_end(df)

In [ ]:
# @title
# len(test[test['ID'] == list_of_IDs[0][0]])/48

In [ ]:
#@title
# df = test

In [ ]:
#@title
# def all_gaps(df):
#   deltas = df['Datetime'].diff()[1:]
#   gaps = deltas[deltas > timedelta(minutes=60)]
#   gaps_result = []
#   # print(f'{len(gaps)} gaps with average gap duration: {gaps.mean()}')
#   for i, g in gaps.iteritems():
#       gaps_result.append(df.iloc[i - 1])
#       # print(f'Start: {datetime.strftime(gap_start, "%Y-%m-%d %H:%M:%S")} | '
#       #       f'Duration: {str(g.to_pytimedelta())}')
      
#   return gaps_result

In [ ]:
#@title
# def find_all_gaps(df):
#   results = {}
#   ID_list = df['ID'].unique().tolist()
#   for index, ID in enumerate(ID_list):
#     clear_output(wait=True)
#     print(index,'/',len(ID_list))
#     ID_v = df[df['ID'] == ID]
#     ID_view = ID_v.sort_values(by=['Datetime'])
#     ID_view.reset_index(drop=True,inplace=True)

#     gaps = all_gaps(ID_view)
#     results[ID] = gaps

#   return results

In [ ]:
#@title
# gaps = find_all_gaps(df)

In [ ]:
#@title
# gap_list = list(gaps.keys())

In [ ]:
#@title
# len(gap_list)

In [ ]:
#@title
# gaps

In [ ]:
#@title
# d1 = pd.to_datetime(gaps[gap_list[0]][0]['Datetime'])


In [ ]:
#@title
#  1101: 12243   01:30:00
# (df[df['ID'] == 1101].sort_values(by=['Datetime']))[pd.to_datetime(gaps[1101][0]['Datetime'])]
# t = df[df['Datetime'] > d1 - timedelta(hours=3) ]
# t = t[t['Datetime'] < d1 + timedelta(hours=3) ]
# print(t[t['ID'] == 1378])

In [ ]:
#@title
# temp3 = []
# for i in gap_list:
#   for j in gaps[i]:
#     temp = df[(df["ID"] == j.ID) ]
#     temp2 = temp[temp.Datetime == j.Datetime ].index[0]
#     # print(temp2)
#     temp3.append(int(temp2))
# for i in temp3:
#   print(i)
#   print(df.iloc[i])

In [ ]:
#@title
# print(df.iloc[12290:12293])
# print(df.iloc[12290])


## Normalization and SAX Implementation

In [ ]:
def normalization(column):
  len_c = len(column)
  iter = int( len_c/48 )

  for i in range(iter):

    x = i*48
    y = (i+1)*48 + 1
    max = column[x:y].max()
    min = column[x:y].min()
    column[x:y] = (column[x:y] - min) / (max - min)

    clear_output(wait=True)
    print(i,"/",iter)
  return column


In [ ]:
# normalized_column = normalization(df.Energy)
# normalized_column.to_csv('normalizedfinal.csv') 

In [ ]:
!gdown --id 1Z1nPYnL3muEr39Ofd2xlrEkJx1OKNVOY
!unzip -q "normalizedfinal.zip" -d ""

Downloading...
From: https://drive.google.com/uc?id=1Z1nPYnL3muEr39Ofd2xlrEkJx1OKNVOY
To: /content/normalizedfinal.zip
243MB [00:02, 90.5MB/s]


In [ ]:
normalized = pd.read_csv(r'/content/normalizedfinal.csv')
df['Energy'] = normalized['Energy']
df.head()

,Datetime,ID,Energy
32,2009-07-18 00:30:00,1392,0.120863
33,2009-07-18 01:00:00,1392,0.197606
34,2009-07-18 01:30:00,1392,0.294814
35,2009-07-18 02:00:00,1392,0.291403
36,2009-07-18 02:30:00,1392,0.513106


In [ ]:
# df['Energy'] = normalized_column            # Saving Values to original DataFrame 

In [ ]:
 df[df['Energy'] == normalized['Energy'].max()]

,Datetime,ID,Energy
73,2009-07-18 21:00:00,1392,1.0
140,2009-07-20 06:30:00,1392,1.0
166,2009-07-20 19:30:00,1392,1.0
281841,2009-07-21 03:00:00,1392,1.0
281876,2009-07-24 09:00:00,1392,1.0
...,...,...,...
13202293,2010-04-24 16:30:00,1253,1.0
13202327,2010-04-25 09:30:00,1253,1.0
13202379,2010-04-23 23:00:00,1253,1.0
13202422,2010-04-27 08:30:00,1253,1.0


In [ ]:
# normalized = pd.read_csv('/content/drive/My Drive/Summer Work and SPROJ/Unzipped/normalized.csv')
# df['Energy'] = normalized['Energy']
# df.head()

In [ ]:
dict_states = {'a':[0.00,0.25],
               'b':[0.25,0.5],
               'c':[0.5,1.0]}

In [ ]:
#@title


# def SAX(state_dict,discrete_data):
#   word = ""
#   keylist = state_dict.keys()  
#   for i in discrete_data:
#     for j in keylist:
#       if state_dict[j][0] <= i < state_dict[j][1]:
#         word += j

#   return word  

# def PAA(cont_data,frame_width):
#   new_discrete_table = []
#   n = len(cont_data)
#   w = int(n/frame_width)
#   # print("n-",n)
#   # print("w-",w)
#   result = []

#   for i in range(0,w):
#     j = int((n/w*(i))+1)-1
#     k = int((n/w)*(i+1))
#     # print("j-",j,"k-",k)
#     sum = 0

#     for l in range(j,k):
#       # print("l-",l)
#       sum = sum + cont_data[l]
#     c = (w/n)*sum
#     for l in range(j,k):
#       new_discrete_table.append(c)
#     result.append(c)
  
#   # for i in range(w-1):
#   #   temp = np.full((1,frame_width),result[i]).T
#   #   new_discrete_table.extend(temp[0])

#   # for i in range(n-(w*frame_width)):
#   #   temp = np.full((1,frame_width),None).T
#   #   new_discrete_table.extend(temp[0])

#   return new_discrete_table, result

In [ ]:
#@title
# temp = df[df["ID"]== df['ID'].iloc[0]]                   #First ID from the list_of_IDs stored in temp variable
# temp.sort_values(by=['Datetime'],inplace=True)            #Sorting DataFrame on Datetime
# temp.reset_index(drop=True,inplace=True)                  #Fixing Index to go from 0,1 ... n.
# PAA_values_graph,PAA_result  = PAA(temp["Energy"],16)     #Running PAA on 48/3 = 16 width dividing a single day into 3.

In [ ]:
#@title
# temp['PAA'] = PAA_values_graph

In [ ]:
#@title
# fig = go.Figure()

# #With PAA running
# # fig.add_trace(go.Scatter(x=temp['Datetime'], y=temp['PAA'],
# #                     mode='lines',
# #                     name='After SAX implementation'))

# #Orignal Data only Normalized.
# fig.add_trace(go.Scatter(x=temp['Datetime'], y=temp['Energy'],
#                     mode='lines',

#                     name='Data(normalized)'))


# fig.show()

In [ ]:
#@title
# fig = go.Figure()

# #With PAA running
# fig.add_trace(go.Scatter(x=temp['Datetime'], y=temp['PAA'],
#                     mode='lines',
#                     name='After SAX implementation'))

# #Orignal Data only Normalized.
# fig.add_trace(go.Scatter(x=temp['Datetime'], y=temp['Energy'],
#                     mode='lines',

#                     name='Data(normalized)'))


# fig.show()

In [ ]:
#@title
# np.mean(temp['Energy'][16*0:16*1])

In [ ]:
#@title
# temp['PAA'][16*0:16*1+1]

In [ ]:
#@title
# SAX(dict_states,PAA_result)

In [ ]:
#@title
# len("abbaaaaacaaaaaaaababaabaabbaabaabaaaaaaaababbaaaaaaabbaaaaaaaabaaaaaaaaaaaaaababaabbaaaaabaabaaaaaaaaaaaaaaaaabaaaaaaaaaaaaaabaaaaaaaaaaaaaaaaaaaaaaaaaaaaabaaaaabaabaaaaabaaaaaaaaaabaaaaaabaabaaaaaaaaaaaaaaaaababbaaaabaaaaabbabaaaaaaaaaaaaaaaaabaabbabbaabaabaababbaaaaabaabaababaaaaaaaaababbaabaabaabaaaaabaabaaaaaaaabaababbaaaaaaabaabbaabaacaabaaaabbabbabbaabaababbabaaababbabbabcaababbabcaabaacaababbabbaababbaababbabbaccaccaacabbaabaababbabcaababcaababbaacaababbaababbaacabbabaabbabcabbabcab")

In [ ]:
#@title
# ts = pd.to_datetime('7/18/2009')
# ts2 = pd.to_datetime('7/16/2009')
# temp2 = temp.loc[temp.Datetime >= ts2]
# temp2.loc[temp.Datetime <= ts]

In [ ]:
def SAX_RUN(cont_dataframe,frame_width):
  new_discrete_df = pd.DataFrame(columns=['Datetime_Range', 'ID', 'Letter','PAA'])
  ID_list = cont_dataframe['ID'].unique().tolist()
  new_discrete_list = []
  result_dict = {}
  full_dict = {}


  for index, ID in enumerate(ID_list):
    temp_discrete_df = pd.DataFrame(columns=['Datetime_Range', 'ID', 'Letter','PAA'])
    # clear_output(wait=True)
    # print(index,'/',len(ID_list))
    ID_v = cont_dataframe[cont_dataframe['ID'] == ID]
    ID_view = ID_v.sort_values(by=['Datetime'])
    ID_view.reset_index(drop=True,inplace=True)
    new_row = {}
    cont_data = ID_view['Energy']
    dtr = []
    result_c = []
    result_dtr = []
    result_letter = []
    result_ID = []

    word = ""

    n = len(cont_data)
    w = int(n/frame_width)
    count = 0

    for i in range(0,w):
      j = int((n/w*(i))+1)-1
      k = int((n/w)*(i+1))

      sum = 0
      for l in range(j,k):
        sum = sum + cont_data[l]
      c = (w/n)*sum
      letter = SAX(dict_states,[c])

      for l in range(j,k):
        result_c.append(c)
        result_dtr.append(ID_view['Datetime'][j])
        result_letter.append(letter)
        result_ID.append(ID)
        
      word += letter
      

    
    result_dict[ID] = word

    temp_discrete_df['Datetime_Range'] = result_dtr
    temp_discrete_df.Letter = result_letter
    temp_discrete_df.PAA = result_c
    temp_discrete_df.ID = result_ID
    new_discrete_df = pd.concat([new_discrete_df,temp_discrete_df])
    
    
  return new_discrete_df, result_dict


In [ ]:
SAX_results_with_dates, SAX_results = SAX_RUN(df,16)

In [ ]:
SAX_results_with_dates[SAX_results_with_dates['ID'] == list_of_IDs.iloc[0]]

,Datetime_Range,ID,Letter,PAA
0,2009-07-18 00:30:00,1392,a,0.221375
1,2009-07-18 00:30:00,1392,a,0.221375
2,2009-07-18 00:30:00,1392,a,0.221375
3,2009-07-18 00:30:00,1392,a,0.221375
4,2009-07-18 00:30:00,1392,a,0.221375
...,...,...,...,...
3835,2009-10-05 16:30:00,1392,c,0.504834
3836,2009-10-05 16:30:00,1392,c,0.504834
3837,2009-10-05 16:30:00,1392,c,0.504834
3838,2009-10-05 16:30:00,1392,c,0.504834


In [ ]:
SAX_results

{1000: 'caaaaaaaaaaaaaaaaaaaabbbababaabaaabaaaabababababaabaabbccbbaaababaaaabacbbbaaaabaaabaaaabaabbabbabbaaaaaaaaaaabaaaaaaaabaabaabaabaabaabaabaabbabbacaabaababcaacaabaabbbbabcbaaaaaaaaaabbbabbaababcaacbcabcabcaabbaaabccaaaaaabcbbaabaabaabaababbaabaabaabaabaabaabaabaabaababbaabaacabcabbaabaababbabaaabaabaaababaaaaabbaaaaaaaaaaaaaabaaaaaaaaaaaabaaaabbbaaccabaaaaaaabbbaaaabbaabbaaaaaaabbaaaaabaacbbbaacaacababcbcacbacbacbbbbbcbababaabaabaaaabbabbcbccbabacbcaaaaabbababaabbaabaaaabaaabaaaaaaaaabaaaaaaaaabaabaabaaaaaaaaaaabaabaaaaaaaaaaaabaabaaaaabcbbcbaaaabaababaaaabbaaaababbabbaaaabbacaaaaabbabbabbabbaabaabaabaabaabaabaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaabaababaaaaaaaaaaabbbaacaaaaaaaaaaaaaaaabababaaaabaaaabbbbbbbbabbaaaaaaaabaaaaabaaaaaaaaaababbabaabbbaaaababaaaabbaababbaabaaaababbabbaaaabbabbaaaaababbaabaaaaabbbaaccbbccaaabbabaaabbbbabbaabbbbaaaacabcbbbaacaababaaacaaabbaaaacbaabaaaaaaaaaaaaaaaaaaabaabaababaaaaaabaabbbbabaabbbabbabbbabbaababaabbaabaabaaabaabbaaaabaaaabaaabaaaaaba

In [ ]:
#@title
# fig_1077 = SAX_results_with_dates

In [ ]:
#@title
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=fig_1077['Datetime_Range'], y=fig_1077['PAA'],
#                     mode='lines',
#                     name='SAX'))
# fig.add_trace(go.Scatter(x=df[df['ID'] == 1077]['Datetime'], y=df[df['ID'] == 1077]['Energy'],
#                     mode='lines',
#                     name='Only Normalized'))
# # fig.add_trace(go.Scatter(x=random_x, y=random_y2,
# #                     mode='markers', name='markers'))

# fig.show()

In [ ]:
#@title
# SAX_results[1077]

In [ ]:
#@title
# len("abbabaabaa")

In [ ]:
#@title
# SAX_results_with_dates[list_of_IDs[0][0]]

In [ ]:
#@title
# SAX_df = pd.DataFrame(list(SAX_results_with_dates.items()),columns = ['ID','Data']) 

In [ ]:
#@title
# SAX_df['ID'][0]

In [ ]:
#@title
# ID_temp = df[df['ID']==SAX_df['ID'][0]]
# ID_temp.sort_values(by=['Datetime'],inplace=True)
# fig = px.line(x=ID_1392['Datetime'], y=ID_1392['Energy'], labels={'x':'DateTime', 'y':'Energy'})

In [ ]:
#@title
# datetime_temp = SAX_df['Data'][0]['Datetime_Range']
# print(len(datetime_temp))

In [ ]:
#@title
# SAX(dict_states , [SAX_df['Data'][19]['letter_list'][10]])
# energy_temp = SAX_df['Data'][0]['letter_list']

In [ ]:
#@title
# print(len(energy_temp))

In [ ]:
#@title
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=datetime_temp, y=energy_temp,
#                     mode='lines',
#                     name='SAX'))
# fig.add_trace(go.Scatter(x=ID_temp['Datetime'], y=ID_temp['Energy'],
#                     mode='lines',
#                     name='Only Normalized'))
# # fig.add_trace(go.Scatter(x=random_x, y=random_y2,
# #                     mode='markers', name='markers'))

# fig.show()

## Time Based Markov Model
**Ft Matrix**

In [ ]:
def Ft_Matrix(symbolic_rep,states,n_states):
    ft_matrix = np.zeros((n_states,n_states))
    for state in states:
        for neighbor in states:
            count = 0
            for i in range(len(symbolic_rep)):
                if i == len(symbolic_rep) - 1:
                    break
                elif (symbolic_rep[i]== state and symbolic_rep[i+1]==neighbor):
                    count = count+1
            state_index = states.index(state)
            neighbor_index = states.index(neighbor)

            ft_matrix[state_index][neighbor_index] = count
    return ft_matrix

**Transition Probability Matrix Pt**

In [ ]:
def pt_matrix_gen(symbolic_rep,states,n_states):  
  ft_matrix = Ft_Matrix(symbolic_rep,states,n_states)
  Pt_matrix = np.zeros((n_states,n_states))
  for i in range(n_states):
      for j in range(n_states):
          total = 0
          for k in range(n_states):
              total = total + ft_matrix[k][j]
          if total == 0:
              Pt_matrix[i][j] = 0
          else:
              Pt_matrix[i][j] = ft_matrix[i][j]/total
  return Pt_matrix

# #  Probability of a state to occur should be equal to 1 - sum of a col must be 1
# ​# print(pt_matrix_gen('abcabbcaaacabaabbaccbbbcbabb'))

In [ ]:
def initialization(dict_states):
    states = list(dict_states.keys())
    states.sort()
    n_states = len(states)
    return states,n_states


In [ ]:
def time_based_MM(dict_states):
    MM_results = {}
    states,n_states = initialization(dict_states)
    for key,value in SAX_results.items():
        ID = key
        symbolic_rep = value
        MM_results[ID] = pt_matrix_gen(symbolic_rep,states,n_states)
    return MM_results


In [ ]:
MM_results = time_based_MM(dict_states)

In [ ]:
SAX_results[1015]

In [ ]:
MM_results[1015]

In [ ]:
# def distance(matrix1, matrix2):
#   return (rel_entr(matrix1,matrix2) + rel_entr(matrix2,matrix1)) / 2

In [ ]:
# def distance_run(matrix_dict):
#   first_dict = {}
#   for i in matrix_dict.keys():
#     first_dict[i] = {}
#     for j in matrix_dict.keys():
#       first_dict[i][j] = distance(matrix_dict[i],matrix_dict[j])

#   return first_dict

In [ ]:
# distance_matrix  = distance_run(MM_results)

In [ ]:
# distance_matrix[1392][1981]

##KL Distance

In [ ]:
_,N = initialization(dict_states)

In [ ]:
def kld_calculation(pi,pj):
    total = 0
    for m in range(N):
        for n in range(N):
            value = pi[m][n] * np.log(pi[m][n]/pj[m][n])
            if np.isfinite(value):
                total = total + value
    kld = (1/N) * total
    
    return kld

In [ ]:
def distance_cal(pi,pj):
    distance = (kld_calculation(pi,pj) + kld_calculation(pj,pi))/2
    return distance

**Converting the index of the points**

In [ ]:
index_dict = {}
i = 1
for key in MM_results.keys():
    index_dict[key] = i
    i = i + 1

# print(index_dict)

**Running Distance Function on Markov Model Results**

In [ ]:
def KL_Dist(matrix_dict):
    first_dict = {}
    for i in matrix_dict.keys():
        first_dict[index_dict[i]] = {}
        for j in matrix_dict.keys():
            first_dict[index_dict[i]][index_dict[j]] = distance_cal(matrix_dict[i],matrix_dict[j])

    return first_dict

In [ ]:
KL_results = KL_Dist(MM_results)

In [ ]:
data = pd.DataFrame(KL_results)
data.head()

**Saving the results in a file (tab separated format)**

In [ ]:
file = open('Updated_dist.txt',"a")
for key, value in KL_results.items():
    for k,v in value.items():
        line = str(key) + '\t'+str(k) +'\t'+str(v)+'\n'
        file.write(line)

## CFSFDP Algo

In [ ]:
!{sys.executable} -m pip install Dcluster

In [ ]:
import Dcluster as dcl
import time

In [ ]:
def cfsfdp_algo(filename):
  start = time.time()
  file=filename
  sep='\t'
  (dist,xxdist,ND,N) = dcl.readfile(file, dimensions = 2, sep=sep)
  XY, eigs = dcl.mds(dist)
  (rho,delta,ordrho,dc,nneigh) = dcl.rhodelta(dist, xxdist, ND, N, percent = 1.0)
  f, axarr = plot1(rho, delta)
  f.show()
  nneigh = nneigh.astype('int')
  rhomin = rho.mean()
  deltamin = delta.mean()
  print('Cutoff: (min_rho, min_delta): (%.2f, %.2f)' %(rhomin,deltamin))
  NCLUST = 0
  cl = np.zeros(ND)-1
  # 1000 is the max number of clusters
  icl = np.zeros(1000)
  for i in range(ND):
      if rho[i]>rhomin and delta[i]>deltamin:
          cl[i] = NCLUST
          icl[NCLUST] = i
          NCLUST = NCLUST+1

  print('NUMBER OF CLUSTERS: %i'%(NCLUST))
  print('Performing assignation')
  # assignation
  # print(cl)
  for i in range(ND):
      if cl[ordrho[i]]==-1:
          cl[ordrho[i]] = cl[nneigh[ordrho[i]]]

  #halo
  # cluster id start from 1, not 0
  ## deep copy, not just reference
  halo = np.zeros(ND)
  halo[:] = cl
  cl = cl.astype('int')
  if NCLUST>1:
      bord_rho = np.zeros(NCLUST)

      for i in range(ND-1):
          for j in range((i+1),ND):
              if cl[i]!=cl[j] and dist[i,j]<=dc:
                  rho_aver = (rho[i]+rho[j])/2
                  if rho_aver>bord_rho[cl[i]]:
                      bord_rho[cl[i]] = rho_aver
                  if rho_aver>bord_rho[cl[j]]:
                      bord_rho[cl[j]] = rho_aver
      for i in range(ND):
          if rho[i]<bord_rho[cl[i]]:
              halo[i] = -1

  for i in range(NCLUST):
      nc = 0
      nh = 0
      for j in range(ND):
          if cl[j]==i:
              nc = nc+1
          if halo[j]==i:
              nh = nh+1
      print('CLUSTER: %i CENTER: %i ELEMENTS: %i CORE: %i HALO: %i'%( i+1,icl[i]+1,nc,nh,nc-nh))
  # print , start from 1

  ## save CLUSTER_ASSIGNATION
  print('Generated file:CLUSTER_ASSIGNATION')
  print('column 1:element id')
  print('column 2:cluster assignation without halo control')
  print('column 3:cluster assignation with halo control')
  clusters = np.array([np.arange(ND)+1,cl+1,halo+1]).T
  np.savetxt('CLUSTER_ASSIGNATION_%.2f_%.2f_.txt'%(rhomin,deltamin),clusters,fmt='%d\t%d\t%d')
  print('Result are saved in file CLUSTER_ASSIGNATION_%.2f_%.2f_.txt'%(rhomin,deltamin))
  # print('\n\nDrag the mouse pointer at a cutoff position in figure DECISION GRAPH and press   OR   Press key n to quit')
  ################# plot the data points with cluster labels
  cmap = cm.rainbow(np.linspace(0, 1, NCLUST))
  plot2(axarr,rho, delta,cmap,cl,icl,XY,NCLUST)
  # f.show()
  f.savefig('Decision_graph_Clustering_(%.2f_%.2f_).png'%(rhomin,deltamin), dpi=300)
  f.show()
  end = time.time()
  print("Time taken: ",end-start)

In [ ]:
%matplotlib notebook
try:
    input = raw_input
except NameError:
    pass

def plot1(rho, delta):
    
    f, axarr = plt.subplots(1,3,figsize=(20,10))
    axarr[0].set_title('DECISION GRAPH')
    axarr[0].scatter(rho, delta, alpha=0.6,c='grey')
    axarr[0].set_xlabel(r'$\rho$')
    axarr[0].set_ylabel(r'$\delta$')
    axarr[1].set_title('DECISION GRAPH 2')
    axarr[1].scatter(np.arange(len(rho))+1, -np.sort(-rho*delta), alpha=0.6,c='grey')
    axarr[1].set_xlabel('Sorted Sample')
    axarr[1].set_ylabel(r'$\rho*\delta$')
    return(f,axarr)


def plot2(axarr,rho, delta,cmap,cl,icl,XY,NCLUST):
    icl = icl.astype('int')
#     c=color.reshape(1,-1)
    axarr[0].scatter(rho, delta, alpha=1,c='grey')
    axarr[1].scatter(np.arange(len(rho))+1, -np.sort(-rho*delta), alpha=1,c='grey')
    for i in range(NCLUST):
        axarr[0].scatter(rho[icl[i]], delta[icl[i]], alpha=0.8, c=cmap[i].reshape(1,-1))
        axarr[1].scatter(i+1, rho[icl[i]]*delta[icl[i]], alpha=0.8,c=cmap[i])
    axarr[2].set_title('2D multidimensional scaling')
    axarr[2].scatter( XY[:,0],  XY[:,1],alpha=0.8,c=cmap[list(cl)])
    axarr[2].set_xlabel('X')
    axarr[2].set_ylabel('Y')



In [ ]:
cfsfdp_algo("Updated_dist.txt")

## Distributive Algorithmn

In [ ]:
dfr = df.copy()
dfr = dfr.dropna()
dfr_split = np.array_split(df, 50)
# split = dfr_split[0]
# split.head()
clusters_lists = []
len(dfr_split[0])

In [ ]:
def k_means(test):
  global clusters_lists
  Y = test['Energy'].values.reshape(-1,1)
  kmeans_2 = KMeans(n_clusters=15, random_state=0).fit(Y)

  closest, _ = pairwise_distances_argmin_min(kmeans_2.cluster_centers_, Y)

  clusters = kmeans_2.cluster_centers_
  labels = kmeans_2.labels_
  Y_dist = kmeans_2.transform(Y)**2
  test['labels'] = labels
  df_New = pd.DataFrame(Y_dist.sum(axis=1).round(2), columns=['sqdist'])
  test['Sq.Dist'] = df_New['sqdist']
  
  clusters_lists.append(clusters)

  return test,closest,clusters

  

In [ ]:
def cls_center(test, closest):
  index = 0
  final_df = pd.DataFrame()
  for i in closest:
    if i == 0:
      new_dfr = test[test['Energy']==test['Energy'].values[i]]
      final_dfr = new_dfr
      index = 1
    else:
      new_sub = test[test['Energy']==test['Energy'].values[i]]
      df_list = [final_df,new_sub]
      final_df = pd.concat(df_list)

  return final_df

  

In [ ]:
def adding_cls_centers(final_df,clusters):
  conditions = []
  for i in range(0,15):
    x = final_df['labels'] == i
    conditions.append(x)
  res = np.select(conditions, clusters, 'Other')
  # conditions
  res
  final_df['Cls.Center'] = res
  final_df.head()

  return final_df

  

In [ ]:
# test['Sq.Dist'].describe()

In [ ]:
def defining_threshold(test):
  threshold = test['Sq.Dist'].min() + test['Sq.Dist'].min()*0.10
  # test[test['Sq.Dist'] < threshold].count()\
  violating_test = test[test['Sq.Dist'] < threshold]
  violating_test_cls = violating_test['labels'].unique()
  Nv = len(violating_test_cls)
  return threshold, violating_test, violating_test_cls, Nv

  

In [ ]:
def adaptive_kmeans(violating_test_cls,violating_test,final_df,test):
  global clusters_lists

  index =14
  for i in violating_test_cls:
    # i = violating_test_cls[2]
    subdata = violating_test[violating_test['labels'] == i]
    sub_data = subdata['Energy'].values.reshape(-1,1)
    try:
      kmeans_sub = KMeans(n_clusters=2,random_state=0).fit(sub_data)
      cluster = kmeans_sub.cluster_centers_
      clusters_lists.append(cluster)
      labels_sub = kmeans_sub.labels_
      sub_data_dist = kmeans_sub.transform(sub_data)**2
      subdata['labels'] = labels_sub
      # df_New = pd.DataFrame(sub_data_dist.sum(axis=1).round(2), columns=['sqdist'])
      subdata['Sq.Dist'] = sub_data_dist


      index = index+1
      subdata['labels'].replace({0 : index,1: index+1},inplace = True)

      closest, _ = pairwise_distances_argmin_min(kmeans_sub.cluster_centers_, sub_data)
      

      for i in closest:
        new_sub = subdata[subdata['Energy']==subdata['Energy'].values[i]]
        df_list = [final_df,new_sub]
        final_df = pd.concat(df_list)
      
      
      final_df['Cls.Center'] = np.where(final_df['labels'] == index, cluster[0],final_df['Cls.Center'])
      final_df['Cls.Center'] = np.where(final_df['labels'] == index+1, cluster[1],final_df['Cls.Center'])
      
      # new_column = pd.Series(subdata['labels'], name='labels', index=subdata.index.values)
      # final_df.update(new_column)
      # new_column = pd.Series(subdata['Sq.Dist'], name='Sq.Dist', index=subdata.index.values)
      # final_df.update(new_column)
      
      new_column = pd.Series(subdata['labels'], name='labels', index=subdata.index.values)
      test.update(new_column)
      new_column = pd.Series(subdata['Sq.Dist'], name='Sq.Dist', index=subdata.index.values)
      test.update(new_column)

      index = index + 1
    except:
      print(i, 'CLUSTER')

  return final_df, test



In [ ]:
df_list = []
for split in dfr_split:
  split = split.dropna()
  test,closest,clusters = k_means(split)
  final_df = cls_center(test, closest)
  updated_final_df = adding_cls_centers(final_df,clusters)
  threshold, violating_test, violating_test_cls, Nv = defining_threshold(test)
  k = 15
  Kmax = 100
  if Nv == 0:
    print(k,' Clusters found')
  elif k+Nv > Kmax:
    print('Clusters did not converge')
  else:
    print(k+Nv, ' Current clusters')
  if Nv== 0:
    df_list.append(final_df)
  else:
    resulting_df,resulting_test = adaptive_kmeans(violating_test_cls,violating_test,updated_final_df,test)

    resulting_df.drop(columns=['Sq.Dist'], inplace= True)
    df_list.append(resulting_df)
  # print('-----------------------------Next Split--------------------------------')
# print(type(df_list[0]))

In [ ]:

final_combined = pd.DataFrame()
for i in df_list:
  final_combined = final_combined.append(i)

final_combined.tail()



## Applying SAX, MM Model and CFSFDP Algo

In [ ]:
ID_list = final_combined['ID']
print(len(ID_list),' Entries in final_df')
print(len(dfr['ID']),' Entires in the whole data')


In [ ]:
distinct_ID_pts = set(ID_list.values)
# distinct_ID_pts

In [ ]:
Sub_SAX_results = {}
for ID_pt in distinct_ID_pts:
  Sub_SAX_results[ID_pt] = SAX_results[ID_pt]

# Sub_SAX_results[1942]

In [ ]:
Sub_MM_results = time_based_MM(dict_states,Sub_SAX_results)
# Sub_MM_results[1942]

In [ ]:
sub_index_dict = {}
i = 1
for key in Sub_MM_results.keys():
    sub_index_dict[key] = i
    i = i + 1


In [ ]:
_,N = initialization(dict_states)
N

In [ ]:
Sub_KL_results = KL_Dist(Sub_MM_results,sub_index_dict)
# Sub_KL_results

In [ ]:
file = open('Subdata_dist.txt',"a")

for key, value in Sub_KL_results.items():
    for k,v in value.items():
        line = str(key) + '\t'+str(k) +'\t'+str(v)+'\n'
        file.write(line)

In [ ]:
cfsfdp_algo("Subdata_dist.txt")